In [25]:
#| export
from win32com import client as wc
from pathlib import Path
from contextlib import contextmanager
import json

In [26]:
#| export
with open("action_parameterset.json", "r", encoding='utf-8') as f:
    action_info = json.load(f)

In [27]:
#| export
def get_absolute_path(path):
    name = Path(path)
    return name.absolute().as_posix()

In [28]:
#| export
class _Actions:
    def __init__(self, HwpApi):
       self._HwpApi = HwpApi
    
    def __getattr__(self, action_key):
        return self._HwpApi.create_action(action_key)
    
    def __dir__(self):
        return [key for key in action_info.keys()]

In [29]:
#| export
class HwpApi:
    def __init__(self, api=None):
        if not api:        
            api = wc.gencache.EnsureDispatch("HWPFrame.HwpObject")
        self.api = api
        self.actions = _Actions(self)
        
                
    def set_visible(self, is_visible=True, window_i=0):
        self.api.XHwpWindows.Item(window_i).Visible = is_visible
    
    
    def get_filepath(self):
        doc = self.api.XHwpDocuments.Active_XHwpDocument
        return doc.FullName
    
    def open(self, path:str):
        name = get_absolute_path(path)
        self.api.Open(name)
        return name
    
    def save(self, path=None, save_format="HWP"):
        if not path:
            self.api.Save()
            return self.get_filepath()
        name = get_absolute_path(path)
        self.api.SaveAs(name, save_format)
        return name
    
    def close(self):
        self.api.Run("Close")
        
    def create_action(self, action_key:str):
        """"""
        return _Action(self, action_key)
    
    def create_parameterset(self, action_key:str):
        """get parameterset of certain action"""
        pset_key, description = action_info.get(action_key, None)
        if not pset_key:
            return None
        return getattr(self.api.HParameterSet, f"H{pset_key}")
    

In [30]:
#| export
class _Action:
    
    def __init__(self, hwpapi:HwpApi, action_key:str):
        
        pset_key, description = action_info.get(action_key, None)
        
        self.action_key = action_key
        self.pset_key = pset_key
        self.description = description
        self.hwpapi = hwpapi
        
        # create action and set
        self.act = hwpapi.api.CreateAction(action_key)
        self._pset = self.act.CreateSet()
        self.act.GetDefault(self._pset)
              
    def __str__(self): return f"<Action {self.action_key}: {self.description}>"

    __repr__ = __str__
    
    def apply_pset(self, pset):
        
        self._pset.Merge(pset.HSet)
        return self
    
    def set_parameter(self, key:str, value):
        
        self._pset.SetItem(key, value)
        return self
    
    def get_parameter(self, key:str):
        
        return self._pset.Item(key)
    
    def run(self):
        self.act.Execute(self._pset)
        return self
    
    def create_pset(self):
        if not self.pset_key:
            return None
        pset = getattr(self.hwpapi.api.HParameterSet, f"H{self.pset_key}")
        self.hwpapi.api.HAction.GetDefault(self.action_key, pset.HSet)
        return pset
        
    @contextmanager
    def pset(self):
        parameterset = self.create_pset()
        yield parameterset
        self.apply_pset(parameterset)
    


In [7]:
hwpapi = HwpApi()
hwpapi.set_visible()

In [8]:
hwpapi.actions.Comment

<Action Comment: 숨은 설명>

In [9]:
act = hwpapi.create_action("InsertText")

In [10]:
pset = act.create_pset()

In [11]:
pset2 = hwpapi.api.HParameterSet.HInsertText

In [12]:
pset3 = hwpapi.create_parameterset("InsertText")

In [13]:
pset.Text = "test1"

In [14]:
pset2.Text = "test2"

In [15]:
pset3.Text = "test3"

In [16]:
for p in [pset, pset2, pset3]:
    act.apply_pset(p).run()

In [17]:
with act.pset() as p:
    p.Text="TextTest"
act.run()

<Action InsertText: 텍스트 삽입>

## ParameterSet Key를 Action Table에서 얻어오기

In [18]:
import pandas as pd

In [19]:
tables = pd.read_html(r"C:\Users\freed\Documents\python_projects\HwpApi\action_table\Action Table_modified.htm")

In [23]:
action_table = pd.DataFrame(tables[1].loc[1:].values, columns=tables[1].loc[0]).query("'Action ID' != `Action ID`")
action_table["ParameterSet"] = action_table["ParameterSet ID"].str.replace("[\+\*\-]", "", regex=True)
action_table["ParameterSet"] = action_table["ParameterSet"].apply(lambda x: x if x else None)
action_table = action_table.loc[~action_table["Action ID"].isin(["VoiceCommand View", "AutoSpellSelect1 ~ 16",])]

In [21]:
pset_keys = action_table[["Action ID", "ParameterSet", "Description"]].set_index("Action ID").apply(lambda row: row.values.tolist(), axis=1).to_dict()

In [22]:
with open("action_parameterset.json", "w", encoding='utf-8') as f:
    json.dump(pset_keys, f)